# Classical Methods

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.utils.data import ConcatDataset, Dataset, DataLoader, random_split

import torch

In [2]:
TRAIN_TEST_SPLIT = (0.8, 0.1) # remaining parts will be test
DIRECTORY_PATH = "/home/mericdemirors/Desktop/TUD_lectures/DLMI/project/project_capsule_dataset"

THRESHOLD = 62859

## Decision Function

In [3]:
class BleedDataset(Dataset):
    def __init__(self, root_dir, image_read_func="RGB"):
        self.root_dir = root_dir
        self.bleeding_dir = os.path.join(root_dir, "bleeding")
        self.healthy_dir = os.path.join(root_dir, "healthy")

        # get image paths
        self.x = [os.path.join(self.bleeding_dir, p) for p in os.listdir(self.bleeding_dir)] + [os.path.join(self.healthy_dir, p) for p in os.listdir(self.healthy_dir)]
        
        # get image labels, bleeding=1, healthy=0
        self.y = [1 for _ in os.listdir(self.bleeding_dir)] + [0 for _ in os.listdir(self.healthy_dir)]
        self.num_samples = len(os.listdir(self.bleeding_dir)) + len(os.listdir(self.healthy_dir))

        # set up the function to use for image reading
        # different reading functions can be written and used with this structure
        if image_read_func == "RGB":
            self.image_read_function = self.read_RGB
        elif image_read_func == "gray":
            self.image_read_function = self.read_gray
        else:
            print("Wrong image_read_func parameter")

    def __len__(self):
        return self.num_samples

    def read_RGB(self, idx):
        path = self.x[idx]
        label = self.y[idx]
        
        image = cv2.imread(path)
        image = image[32:544, 32:544] # cropping image to get rid of the black borders
        image[:48,:48] = [0,0,0] # painting the upper left corner if there is a gray square
        image[:31, 452:] = [0,0,0] # painting the upper right corner if there is white text parts
        image = np.transpose(image, [2,0,1]) # adjust the axises into the pytorch dimensions of [B, C, W, H]

        return image, label
    
    def read_gray(self, idx):
        path = self.x[idx]
        label = self.y[idx]
        
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = image[32:544, 32:544] # cropping image to get rid of the black borders
        image[:48,:48] = 0 # painting the upper left corner if there is a gray square
        image[:31, 452:] = 0 # painting the upper right corner if there is white text parts
        image = image[np.newaxis, ...] # adjust the axises into the pytorch dimensions of [B, C, W, H]
        
        return image, label

    def __getitem__(self, idx):
        image, label = self.image_read_function(idx)
        return image, label

In [4]:
### ---|---|---|---|---|---|---|---|---|---|--- DATASET SPLIT ---|---|---|---|---|---|---|---|---|---|--- ###
dataset = BleedDataset(DIRECTORY_PATH, image_read_func="RGB")

train_size = int(TRAIN_TEST_SPLIT[0] * len(dataset))
test_size = int(TRAIN_TEST_SPLIT[1] * len(dataset))
validation_size = len(dataset) - train_size - test_size

torch.manual_seed(0) # setting the seed to 0 so dataset split is same for every run
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
train_validation_dataset = ConcatDataset([train_dataset, validation_dataset])

train_validation_dataloader = DataLoader(train_validation_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## Creating Decision Function

In [5]:
def torch_tensor_to_RGB_numpy(image):
    image = np.transpose(image, (1, 2, 0))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

def function(image, window_size=18, red_multiplier=5.5, green_multiplier=-9.5, blue_multiplier=-0.5, percentile=2):

    image_rgb = image.astype(np.float32)

    redness_score = image_rgb[:, :, 0] * red_multiplier + image_rgb[:, :, 1] * green_multiplier + image_rgb[:, :, 2] * blue_multiplier

    # remove found spots outside the image
    mask = cv2.imread("mask_meric.png", cv2.IMREAD_GRAYSCALE)

    redness_score = redness_score - np.abs((mask * np.min(redness_score)))

    # Find the top % reddish pixel values
    threshold = np.percentile(redness_score, 100-percentile)
    high_red_indices = np.where(redness_score >= threshold)

    # Initialize variables for the best score and coordinates
    max_score = -np.inf
    best_coords = (0, 0)

    # Calculate window scores only around high redness indices
    h, w = redness_score.shape
    for row, column in zip(*high_red_indices):
        # Define the top-left corner of the window
        top_left_row = max(0, row - window_size // 2)
        top_left_column = max(0, column - window_size // 2)

        # Define the bottom-right corner of the window
        bottom_right_row = min(h, top_left_row + window_size)
        bottom_right_column = min(w, top_left_column + window_size)

        # Extract the window and calculate its redness score
        window = redness_score[top_left_row:bottom_right_row, top_left_column:bottom_right_column]
        score = np.sum(window)

        if score > max_score:
            max_score = score
            best_coords = (row, column)
    
    # Highlight the most reddish area on the image
    image_rgb = image_rgb.astype(np.uint8)
    
    left_upper_column_row = [max(0, best_coords[1] - window_size//2), max(0, best_coords[0] - window_size//2)]
    right_lower_column_row = [min(w, best_coords[1] + window_size//2), min(h, best_coords[0] + window_size//2)]
    cv2.rectangle(image_rgb, left_upper_column_row, right_lower_column_row, (0, 255, 0), 2)


    return max_score, image_rgb

decision_function = function

# Rectangles bleeding annotation from the dataset bounding boxes

In [ ]:
import pandas as pd
path = "/home/mericdemirors/Desktop/TUD_lectures/DLMI/project/project_capsule_dataset/bleeding/image17968.jpg"
df = pd.read_csv("/home/mericdemirors/Desktop/TUD_lectures/DLMI/project/project_capsule_dataset/bboxes_bleeding.csv")

image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

line = df[df["filename"] == os.path.split(path)[1]].values[0][1:]
xywh = [int(x*576) for x in line]

rectangle_image = cv2.rectangle(image, [xywh[0]-xywh[2]//2,xywh[1]-xywh[3]//2], [xywh[0]+xywh[2]//2, xywh[1]+xywh[3]//2], (0, 0, 255), 2)
plt.imshow(rectangle_image)
plt.show()

# Rectangles the found most red part (image is read from path)

In [ ]:
image = cv2.imread(path)
image = image[32:544, 32:544] # cropping image to get rid of the black borders
image[:48,:48] = [0,0,0] # painting the upper left corner if there is a gray square
image[:31, 452:] = [0,0,0] # painting the upper right corner if there is white text parts
image = np.transpose(image, [2,0,1]) # adjust the axises into the pytorch dimensions of [B, C, W, H]
np_image = torch_tensor_to_RGB_numpy(image)
score, annotated_image = decision_function(np_image, *[18, 5.5, -9.5, -0.5, 10])
plt.imshow(annotated_image)

if score < THRESHOLD:
    plt.title(f"Image redness score: {score}, HEALTHY")
else:
    plt.title(f"Image redness score: {score}, BLEEDING")

plt.show()

# Image comes from dataset

In [ ]:
image = train_dataset[443][0]
np_image = torch_tensor_to_RGB_numpy(image)
score, annotated_image = decision_function(np_image, *[18, 5.5, -9.5, -0.5, 10])
plt.imshow(annotated_image)
if score < THRESHOLD:
    plt.title(f"Image redness score: {score}, HEALTHY")
else:
    plt.title(f"Image redness score: {score}, BLEEDING")